Data source: http://www.tse.com.tw/zh/page/trading/exchange/MI_5MINS_INDEX.html
1. before 2011-01-16 is a data per min
2. before 2014-02-23 is a data per 15 sec
3. before 2014-12-29 is a data per 10 sec
4. after 2014-12-29 to today is a data per 5 sec

What we need to complete this shit
1. script1: from 2004-10-15 to today
2. script2: from today keep update data, need to use another script
3. creat a DB with a table contain every columns, different date have different column lenth. (how to make it auto variable with data change?)  (idea: for script1, just drop the fix numbers columns. Then, as script2 insert data, just insert the columns which has time column)  

In [6]:
# -*-coding: utf-8 -*-
import sys
import datetime as dt
import requests
import os
import pandas as pd
import sqlite3
import csv
import time


        
def date_trans(start1,end1):

    date_int={'year1':int(start1[:4]),'month1':int(start1[4:6]),'day1':int(start1[6:]),
              'year2':int(end1[:4]),'month2':int(end1[4:6]),'day2':int(end1[6:])}
    
    start_date = dt.datetime(date_int['year1'],date_int['month1'],date_int['day1'])
    end_date = dt.datetime(date_int['year2'],date_int['month2'],date_int['day2'])
    totaldays = (end_date - start_date).days + 1
    return start_date,totaldays


def get_content(start_date,totaldays):
    
    for daynumber in range(totaldays):
        date1=(start_date + dt.timedelta(days = daynumber)).date()

        five_sec_url='http://www.twse.com.tw/exchangeReport/MI_5MINS?response=csv&date=%s'%str(date1)
        r = requests.get(five_sec_url)
        con = r.text.split('\n')
        print(con)
        res = delIntro(con.encode('utf-8', 'ignore'))
        print(res)
        insertDate(res,str(date1))

    
def delIntro(con):    # delete the introduction of content
    del con[0]
    del con[con.index('"說明:"\r'.encode('utf-8', 'ignore')):]  
    res = [[con[i].strip('=').strip() for i in range(len(con))][j][1:-2].replace(',','').split('\"\"') for j in range(len(con))]
    return res

    
def insertDate(res,date1):    # insert date to first column 
    res[0].insert(0,'日期')
    for i in range(len(res)-1):
        res[i+1].insert(0,date1)

        
def createTable():
    with sqlite3.connect('testDB') as DB:
        c=DB.cursor()
        createTable='''create table if not exists fiveSec (日期 date, 時間 time,累積委託買進筆數 int,  累積委託買進數量 int, 累積委託賣出筆數 int, 累積委託賣出數量 int, 累積成交筆數 int, 累積成交數量 int, 累積成交金額 int);'''
        c.execute(createTable)
        
        
def insertData(res):
    with sqlite3.connect('testDB') as DB:
        c=DB.cursor()
        c.executemany('insert into fiveSec values(?,?,?,?,?,?,?,?,?)',res[1:])
        
def selectData():
    with sqlite3.connect('testDB') as DB:
        df = pd.read_sql_query('select * from fiveSec',DB)
        print(df)

        


# main   
if __name__ =='__main__': 
    
    starttime = time.time()

    createTable()  

    start_date,totaldays=date_trans('20180810','20180810')
#     start_date,totaldays=date_trans(sys.args[0],sys.args[1])
    
    get_content(start_date,totaldays)

    selectData()

    print('cost time:',time.time()-starttime)

['"106年12月08日每5秒委託成交統計"\r', '"時間","累積委託買進筆數","累積委託買進數量","累積委託賣出筆數","累積委託賣出數量","累積成交筆數","累積成交數量","累積成交金額",\r', '="09:00:00","207,305","3,520,054","178,763","1,198,084","0","0","0",\r', '="09:00:05","209,269","3,525,070","180,081","1,211,394","9,451","42,485","2,426",\r', '="09:00:10","211,424","3,647,030","181,541","1,290,906","15,458","69,795","3,188",\r', '="09:00:15","212,715","3,685,581","182,689","1,333,299","18,242","78,173","3,639",\r', '="09:00:20","213,924","3,683,537","183,815","1,338,953","20,189","84,263","3,853",\r', '="09:00:25","215,120","3,691,514","184,778","1,341,560","21,729","89,344","4,022",\r', '="09:00:30","216,460","3,708,762","185,917","1,358,338","22,891","93,398","4,151",\r', '="09:00:35","218,169","3,733,507","187,329","1,372,405","24,046","96,955","4,316",\r', '="09:00:40","219,638","3,739,950","188,547","1,377,534","24,981","100,470","4,433",\r', '="09:00:45","221,367","3,778,028","190,044","1,401,847","25,806","103,470","4,560",\r', '="09:00:50","225,807",

AttributeError: 'list' object has no attribute 'encode'